In [7]:
import os
import numpy as np
import cv2 as cv
from utils import Utils
from sklearn.mixture import GaussianMixture
import time


In [8]:

menPath = "../Dataset_0-5/men/"
womenPath = "../Dataset_0-5/Women/"

In [ ]:
for i in range(1, 10):
    # Load the image
    img = cv.imread(f'../Dataset_0-5/men/2/2_men ({i}).JPG')
    start_time = time.time()
    img_width = 120
    h, w = img.shape[:2]
    new_height = int(h * img_width / w)
    img_size = (img_width, new_height)
    img = cv.resize(img, img_size)    # resize image

    # Convert the image from BGR to YCbCr
    ycbcr_image = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)

    # Extract Cr and Cb channels
    cr = ycbcr_image[:, :, 1]
    cb = ycbcr_image[:, :, 2]

    # Reshape channels into a 1D array
    cr_flat = cr.flatten().reshape(-1, 1)
    cb_flat = cb.flatten().reshape(-1, 1)

    # Concatenate channels into a single array
    x = np.concatenate((cr_flat, cb_flat), axis=1)

    # Fit Gaussian mixture model
    gmm = GaussianMixture(n_components=2)
    gmm.fit(x)

    # Predict probabilities of each pixel belonging to background or foreground
    probs = gmm.predict_proba(x)
    probs_bg = probs[:, 0].reshape(cr.shape)
    probs_fg = probs[:, 1].reshape(cr.shape)

    # Threshold probabilities to extract foreground object from background
    thresh = 0.5
    mask = (probs_fg > thresh).astype(np.uint8) * 255

    # Apply mask to original image
    result1 = cv.bitwise_and(img, img, mask=mask)
    result2 = cv.bitwise_and(img, img, mask=~(mask))

    # Convert image to YCrCb color space
    img_ycrcb1 = cv.cvtColor(result1, cv.COLOR_BGR2YCrCb)
    img_ycrcb2 = cv.cvtColor(result2, cv.COLOR_BGR2YCrCb)

    # Define range of skin color in YCrCb color space
    lower_skin = np.array([0, 135, 85], dtype=np.uint8)
    upper_skin = np.array([255, 180, 135], dtype=np.uint8)

    # Extract skin color from image
    mask1 = cv.inRange(img_ycrcb1, lower_skin, upper_skin)
    mask2 = cv.inRange(img_ycrcb2, lower_skin, upper_skin)

    # Apply morphological operations to remove noise from mask
    kernel = np.ones((3, 3), np.uint8)
    mask1 = cv.erode(mask1, kernel, iterations=1)
    mask1 = cv.dilate(mask1, kernel, iterations=1)

    mask2 = cv.erode(mask2, kernel, iterations=1)
    mask2 = cv.dilate(mask2, kernel, iterations=1)

    # Apply mask to original image to extract hand
    # result1 = cv.bitwise_and(img, img, mask=mask1)
    # result2 = cv.bitwise_and(img, img, mask=mask2)
    result11 = cv.bitwise_and(img, img, mask=mask1)
    result22 = cv.bitwise_and(img, img, mask=mask2)

    mean1 = np.mean(result11)
    mean2 = np.mean(result22)
    # print(mean1, mean2)
    if mean1 > mean2:
        result = result1
    else:
        result = result2
    # Reshape the image to a 2D array of pixels and 3 color values (RGB)
    rows, cols = result.shape[0], result.shape[1]
    image_2d = result.reshape(rows*cols, 3)

    # Perform k-means clustering with 3 clusters using OpenCV
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv.KMEANS_RANDOM_CENTERS
    compactness, labels, centers = cv.kmeans(image_2d.astype(np.float32), 2, None, criteria, 10, flags)

    # Reshape the labels back into the original image shape
    labels_2d = labels.reshape(rows, cols)

    # Get the pixel indices for each label
    indices_0 = (labels_2d == 0)
    indices_1 = (labels_2d == 1)
    # indices_2 = (labels_2d == 2)
    # Get a separate image for each cluster
    result_0 = result.copy()
    result_0[~indices_0] = 0

    result_1 = result.copy()
    result_1[~indices_1] = 0

    # result_2 = result.copy()
    # result_2[~indices_2] = 0
    # io.imshow(result_0)
    # io.show()
    # io.imshow(result_1)
    # io.show()

    mean_0 = np.mean(result_0)
    mean_1 = np.mean(result_1)
    print(mean_0, mean_1)
    if mean_1 > mean_0:
        result_temp = result_1
        indices_temp = indices_1
    else:
        result_temp = result_0
        indices_temp = indices_0
    #  Create a mask for the hand using the pixel indices of one of the clusters
    # mask = np.zeros((rows, cols), dtype=bool)
    # mask[indices_temp] = True

    # # Apply the mask to the original image to get only the hand
    # hand_image = np.zeros_like(result)
    # hand_image[mask] = result[mask]
    end_time = time.time()
    takenTime = end_time - start_time
    # io.imshow(hand_image)
    # io.show()
    cv.imshow('result_temp',  cv.resize(result_temp, (1024, 512)))
    # cv.waitKey(0)
    # cv.imshow('hand_image',  cv.resize(hand_image, (1024, 512)))
    # cv.waitKey(0)
    
    #print taken time in percsion of 2
    print("Time taken: {:.2f} seconds".format(takenTime))

In [11]:
time_taken = 0
img_width = 120
for i in range(1,11):
    # Load the image
    img = cv.imread(f"../Dataset_0-5/men/0/0_men ({i}).JPG")
    h, w = img.shape[:2]
    new_height = int(h * img_width / w)
    img_size = (img_width, new_height)
    img = cv.resize(img, img_size)    # resize image
    # Convert the image to grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Compute the 58ULBP code for the grayscale image
    code = Utils.get_9ULBP(gray)
    # print(type(code))

    print(code)
    print(code.max())

[0.9951824  0.02116471 0.03659732 0.02557403 0.03174707 0.00573211
 0.02910148 0.02072378 0.06966718]
0.9951824
[0.952756   0.15735564 0.16166674 0.10885561 0.0991556  0.03556668
 0.05496669 0.07867782 0.09592227]
0.952756
[0.9586233  0.11969666 0.16589537 0.10499708 0.09974722 0.03569901
 0.06614815 0.07874781 0.07979777]
0.9586233
[0.95942575 0.14144677 0.14254326 0.10197325 0.08662244 0.03399108
 0.08004352 0.08114    0.08442947]
0.95942575
[0.96215767 0.10727891 0.13633361 0.09386905 0.09722151 0.02570224
 0.06928429 0.09051657 0.11063137]
0.96215767
[0.9547946  0.15227807 0.15672414 0.1067058  0.08780998 0.02889949
 0.06446809 0.07002567 0.10781731]
0.9547946
[0.9480657  0.1404112  0.16129881 0.11140062 0.08819216 0.02320846
 0.09631512 0.09863597 0.1253257 ]
0.9480657
[0.94734025 0.1464682  0.14378072 0.12496829 0.09809339 0.02015617
 0.08734342 0.10078087 0.13034327]
0.94734025
[0.956868   0.18914832 0.12556906 0.07152668 0.06516875 0.02384223
 0.03973704 0.07470564 0.12556906]


In [ ]:
# Load image
img = cv.imread('../Dataset_0-5/men/2/2_men (9).JPG')

# Convert image to YCrCb color space
img_ycrcb = cv.cvtColor(img, cv.COLOR_BGR2YCrCb)

# Define range of skin color in YCrCb color space
lower_skin = np.array([0, 135, 85], dtype=np.uint8)
upper_skin = np.array([255, 180, 135], dtype=np.uint8)

# Extract skin color from image
mask = cv.inRange(img_ycrcb, lower_skin, upper_skin)

# Apply morphological operations to remove noise from mask
kernel = np.ones((3, 3), np.uint8)
mask = cv.erode(mask, kernel, iterations=1)
mask = cv.dilate(mask, kernel, iterations=1)

# Apply mask to original image to extract hand
result = cv.bitwise_and(img, img, mask=mask)

cv.imshow('r',  cv.resize(result, (1024, 512)))
cv.waitKey(0)

In [5]:
# Set HOG parameters
win_size = (64, 64)
block_size = (16, 16)
block_stride = (8, 8)
cell_size = (8, 8)
nbins = 9
# Create HOG descriptor
hog = cv.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
for i in range(1,11):
    # Load the image
    img = cv.imread(f"../Dataset_0-5/men/0/0_men ({i}).JPG")
    img_width = 120
    h, w = img.shape[:2]
    new_height = int(h * img_width / w)
    img_size = (img_width, new_height)
    img = cv.resize(img, img_size)    # resize image
    # Convert the image to grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    f = hog.compute(gray)
    print(f)
    print(f.max())
    print("************************************************")

[0.05295024 0.15348765 0.2945944  ... 0.02164842 0.00704529 0.01718291]
0.9041685
************************************************
[0.36488992 0.2491917  0.09057675 ... 0.14185953 0.08116683 0.0228403 ]
0.5263316
************************************************
[0.30413458 0.30413458 0.04647462 ... 0.1307406  0.23544417 0.20929918]
0.529315
************************************************
[0.2040595  0.20958178 0.19479568 ... 0.08541711 0.048785   0.01790663]
0.4478565
************************************************
[0.2928592  0.2928592  0.22016443 ... 0.07346305 0.03220836 0.04575601]
0.55554265
************************************************
[0.1525748  0.23331986 0.20184182 ... 0.12034313 0.19865091 0.19646692]
0.52508473
************************************************
[0.24181023 0.24181023 0.24181023 ... 0.05440654 0.00614775 0.0028718 ]
0.48858282
************************************************
[0.2566191  0.1638082  0.074618   ... 0.18375733 0.05513272 0.00688975]
0.452667
